# 3.1. The `HP` Object and its methods

The `HP` object provides several methods for defining your hyperparameters. These methods allow you to create flexible and type-safe configurations for your machine learning projects.

## Select

The `select` method allows you to choose from a list or dictionary of options:

`hp.select(options, default=None, name=None)`

### Dictionary Options

When using a dictionary, the keys can only be of type `str`, `int`, `bool`, or `float`. The values can be of any type.

Example:

```python
optimizer = hp.select({'adam': torch.optim.Adam, 
                       'sgd': torch.optim.SGD,
                       1: 'custom_optimizer',
                       True: lambda lr: torch.optim.AdamW(lr=lr)},
                      default='adam')
```

In this example, we define a dictionary of optimizers. The keys are of various allowed types (string, integer, boolean), and the values include class references, strings, and even lambda functions.

### List Options

When using a list, the values can only be of type `str`, `int`, `bool`, or `float`. Internally, lists are automatically converted into dictionaries where the keys are the values themselves.

Example:

```python
activation = hp.select(['relu', 'tanh', 'sigmoid'], default='relu')
```

This is equivalent to:

```python
activation = hp.select({'relu': 'relu', 'tanh': 'tanh', 'sigmoid': 'sigmoid'}, default='relu')
```

## Text Input

The `text_input` method defines a text input with an optional default value:

`hp.text_input(default=None, name=None)`

Example:

```python
model_name = hp.text_input(default='my_awesome_model')
```

The value specified in the first argument is the default. In future implementations, this method will allow for runtime type checking to ensure that only string values are provided.

## Number Input

The `number_input` method defines a numeric input with an optional default value:

`hp.number_input(default=None, name=None)`

Example:

```python
learning_rate = hp.number_input(default=0.001)
```

As with `text_input`, the first argument specifies the default value. Future implementations will include runtime type checking to ensure that only numeric values (integers or floats) are provided.

## Complete Example

Here's a comprehensive example showcasing all these methods:

```python
from hypster import HP, config

@config
def hp_methods_example(hp: HP):
    import torch
    
    # Select from a list (internally converted to a dictionary)
    activation = hp.select(['relu', 'tanh', 'sigmoid'], default='relu')
    
    # Select from a dictionary with various key types
    optimizer = hp.select({
        'adam': torch.optim.Adam,
        'sgd': torch.optim.SGD,
        1: 'custom_optimizer',
        True: lambda lr: torch.optim.AdamW(lr=lr)
    }, default='adam')
    
    # Text input
    model_name = hp.text_input(default='my_awesome_model')
    
    # Number input
    learning_rate = hp.number_input(default=0.001)
    
    # Using the selected values
    if activation == 'relu':
        act_func = torch.nn.ReLU()
    elif activation == 'tanh':
        act_func = torch.nn.Tanh()
    else:
        act_func = torch.nn.Sigmoid()
    
    if isinstance(optimizer, str):
        opt = optimizer  # It's a custom optimizer name
    elif callable(optimizer):
        opt = optimizer(learning_rate)  # It's a lambda function
    else:
        opt = optimizer(torch.nn.Linear(10, 10).parameters(), lr=learning_rate)
    
    print(f"Configured {model_name} with {activation} activation and {opt.__class__.__name__} optimizer")

# Usage
result = hp_methods_example(selections={'activation': 'tanh', 'optimizer': 'sgd'})
```

This example demonstrates how to use all the `HP` methods, including selecting from lists and dictionaries with various key types, and using text and number inputs. It also shows how to work with the selected values in your configuration function.